In [1]:
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [32]:
from openai import OpenAI

gpt3_model = "gpt-3.5-turbo-1106"
gpt4_model = "gpt-4-1106-preview"

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Finance Assistant",
    instructions="You are a data scientist for personal Finance assistant. Answer questions briefly, in a sentence or less.",
    model=gpt4_model,
)
show_json(assistant)

{'id': 'asst_OkDJG98KdIwH2VsRjzCvW4HG',
 'created_at': 1705835261,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a data scientist for personal Finance assistant. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Finance Assistant',
 'object': 'assistant',
 'tools': []}

In [33]:
# Upload the file
# https://platform.openai.com/docs/assistants/tools/supported-files
file = client.files.create(
    file=open(
        "data/spending_from_yaml.json",
        "rb",
    ),
    purpose="assistants",
)

In [34]:
# Update Assistant with tools
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id]
)

In [35]:
show_json(assistant)

{'id': 'asst_OkDJG98KdIwH2VsRjzCvW4HG',
 'created_at': 1705835261,
 'description': None,
 'file_ids': ['file-x8RlnUI3KSYMxmP40d94Ga0X'],
 'instructions': 'You are a data scientist for personal Finance assistant. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Finance Assistant',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

## Utils 

In [36]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [37]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(assistant.id, thread, user_input)
    return thread, run

In [38]:
# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [39]:
thread, run = create_thread_and_run(
    "What are those years in my spending data?"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What are those years in my spending data?
assistant: The years in your spending data appear to be 2019, 2020, 2021, 2022, and 2023.



In [40]:
thread, run = create_thread_and_run(
    "Give a stats of spending in 2023, list all spending in 2023 first, then include sum, average of month, max pending amount and month, min pending amount and month in stats."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Give a stats of spending in 2023, list all spending in 2023 first, then include sum, average of month, max pending amount and month, min pending amount and month in stats.
assistant: The spending in 2023 for each month is as follows:

- January: \$7,949.39
- February: \$6,945.28
- March: \$9,967.68
- April: \$15,961.15
- May: \$8,865.40
- June: \$8,839.26
- July: \$10,503.68
- August: \$8,225.97
- September: \$8,239.02
- October: \$10,382.03
- November: \$11,587.64
- December: \$9,306.37

The summary statistics for the spending in 2023 are:
- Total spending: \$116,772.87
- Average monthly spending: \$9,731.07
- Maximum spending was in April with \$15,961.15
- Minimum spending was in February with \$6,945.28【7†source】.



In [41]:
thread, run = create_thread_and_run(
    "Why April has highest spending? Any hint?"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Why April has highest spending? Any hint?
assistant: April's highest spending is due to a "Sydney road trip, 回国机票" which hints at travel costs including a trip to Sydney and plane tickets to return to the home country【0†source】.

